In [ ]:
import os
import requests
import re
from bs4 import BeautifulSoup
import sys

# Function to get the HTML source text of the Medium article
def get_page():
    global url

    # Ask the user to input the URL of a Medium article
    url = input("Enter URL of a Medium article: ")

    # Handling possible error
    if not re.match(r'https?://medium.com/', url):
        print('Please enter a valid Medium article URL.')
        sys.exit(1)

    # Add a User-Agent header to simulate a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    # Call the get method in requests object, pass URL and headers, and collect it in res
    res = requests.get(url, headers=headers)

    # Ensure the request succeeded
    res.raise_for_status()

    # Parse the HTML content
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

# Function to remove all HTML tags and replace some with specific strings
def clean(text):
    rep = {"<br>": "\n", "<br/>": "\n", "<li>": "\n"}
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)
    text = re.sub(r'<.*?>', '', text)
    return text

def collect_text(soup):
    text = f'URL: {url}\n\n'
    para_text = soup.find_all('p')
    print(f"Paragraphs text = \n {para_text}")
    for para in para_text:
        text += f"{para.get_text()}\n\n"
    return text

# Function to save the file in the current directory
def save_file(text):
    if not os.path.exists('./scraped_articles'):
        os.mkdir('./scraped_articles')
    name = url.split("/")[-1]
    fname = f'scraped_articles/{name}.txt'

    # Write a file using with
    with open(fname, 'w', encoding='utf-8') as f:
        f.write(text)

    print(f'File saved in directory {fname}')

# Ensure the script is run directly
if __name__ == '__main__':
    text = collect_text(get_page())
    save_file(text)